In [1]:
!pip install Scikit-image

     |████████████████████████████████| 14.0 MB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 195 kB 96.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 58.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 39.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 77.9 MB/s eta 0:00:01


In [2]:
%matplotlib inline  
import timeit
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from PIL import Image
import skimage
import os
import joblib
from tensorflow.keras import layers, models

In [3]:
CLUSTER_TYPE ='docker_cluster'
%env CLUSTER_TYPE $CLUSTER_TYPE

env: CLUSTER_TYPE=docker_cluster


In [4]:
%%script bash --bg --out script_out

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    echo "Launching scheduler and worker"
    
    HOSTIP=`hostname -I | xargs`
    
    echo "dask-scheduler --host $HOSTIP --dashboard-address $HOSTIP:8787"
    
    # dask scheduler 
    dask-scheduler --host $HOSTIP --dashboard-address $HOSTIP:8787 &

    # dask worker
    dask-worker $HOSTIP:8786 --memory-limit 2GB --nworkers 2 &

fi

In [5]:
host_ip = !hostname -I | xargs
host_ip = host_ip[0]

In [6]:
from dask.distributed import Client

if CLUSTER_TYPE == 'local':
    
    client = Client()

elif CLUSTER_TYPE == 'docker_container':
    
    client = Client('{}:8786'.format(host_ip))
    
elif CLUSTER_TYPE == 'docker_cluster':
    
    # use the provided master
    client = Client('dask-scheduler:8786')
    
client

<Client: 'tcp://192.168.192.2:8786' processes=6 threads=6, memory=11.62 GiB>

In [10]:
faces_path = '/opt/workspace/data/Faces3/'
no_faces_path = '/opt/workspace/data/noFaces3/'


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = skimage.io.imread(os.path.join(folder,filename))
        if img.shape[0]==198 and img.shape[1]==198:
            images.append(img)
    return images

images_with_face = np.array(load_images_from_folder(faces_path))
images_without_face = np.array(load_images_from_folder(no_faces_path))

In [12]:
# def show_image(x, y, index):
#     categ=''
#     plt.imshow(x[index])
#     plt.show()
#     if y[index]==0 : categ='NO FACE'
#     elif y[index]==1 : categ='FACE'
#     else: 'ERROR'   
#     print('label: ', y[index], " -->", categ)
    
# show_image(images_with_face, labels, 0)
# show_image(images_without_face, labels, 1999)

In [13]:
# images[0].shape

### CNN Implementation - Dask 

In [14]:
import dask
from dask.array.image import imread
import dask.array as da

face = da.from_array(images_with_face, chunks=(100,198,198,3))
no_face = da.from_array(images_without_face, chunks=(100,198,198,3))


print('\n Turtles with face, separated in 20 chunks of 100 images each.\n')
display(face)
print('\n Images with no face, separated in 20 chunks of 100 images each.\n')

display(no_face)

#------Concatenate dask arrays--------
arr = []
arr.append(face)
arr.append(no_face)
images = da.concatenate(arr, axis=0) 
print('\n All images\n ')
display(images)



# ------- Create Labels ---------
L = 1999
labels_face = da.from_array(np.ones(L), chunks = 100)
labels_noface = da.from_array(np.zeros(L), chunks = 100)
arr_l = []
arr_l.append(labels_face)
arr_l.append(labels_noface)
labels = da.concatenate(arr_l, axis = 0)
print('\n All labels\n ')
display(labels)


 Turtles with face, separated in 20 chunks of 100 images each.



dask.array<array, shape=(1999, 198, 198, 3), dtype=uint8, chunksize=(100, 198, 198, 3), chunktype=numpy.ndarray>


 Images with no face, separated in 20 chunks of 100 images each.



dask.array<array, shape=(1999, 198, 198, 3), dtype=uint8, chunksize=(100, 198, 198, 3), chunktype=numpy.ndarray>


 All images
 


,Array,Chunk
Bytes,448.43 MiB,11.22 MiB
Shape,"(3998, 198, 198, 3)","(100, 198, 198, 3)"
Count,80 Tasks,40 Chunks
Type,uint8,numpy.ndarray



 All labels
 


dask.array<concatenate, shape=(3998,), dtype=float64, chunksize=(100,), chunktype=numpy.ndarray>

In [33]:
#-----Define training and validation sets------------

perc_training = 0.8
m_training = int(len(images)*0.8)
m_val = len(images) - int(len(images)*0.8)

images_training = images[:m_training]
images_training = images_training.rechunk(chunks=(10,198,198,3))

images_val = images[m_training:m_training+m_val]


labels_training = labels[:m_training]
labels_training = labels_training.rechunk(chunks=10)

lables_val = labels[m_training:m_training+m_val]



#took this from the complicated example for joining images and lables into one variable
#this makes sure the images stay with the correct lables when distributed



# images_training = images_training.to_delayed().flatten().tolist()
# labels_training = labels_training.to_delayed().flatten().tolist()
# batches = [dask.delayed([im, la]) for im, la in zip(images_training, labels_training)]



# #Quick check to see that everything is correct
# print('The lenght of the training data is 3200. Each batch contains 100 images, so the lenght of "batches" should be 32.')
# print(len(batches))
# print('The lenght of the labels of each batch should be equal to the number of images in a batch, so 100.')
# print(len(batches[0][1].compute()))

In [34]:
def build_cnn():

    #filters: how many features/patterns we are going to recognize in the images
    #kernel: shape of the matrix used for each filter's convolution
    #pooling: shape of the matrix used to generalise the feature map created by the convolution (the smaller the shape the longer the computation time)
    
    cnn = models.Sequential([

        layers.Conv2D(filters=30, kernel_size=(3,3), activation='relu', input_shape=(198,198,3)),
        layers.MaxPooling2D((10,10)),
        layers.Conv2D(filters=30, kernel_size=(3,3), activation='relu', input_shape=(198,198,3)),
        layers.MaxPooling2D((10,10)),
        #dense
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dense(2, activation='softmax')
        
    ])
    
    # compile
    cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    #summary
    print(cnn.summary())
    return cnn



def trainFunction(model, weights, epochs, images_val, lables_val):
    queue = c.worker.tensorflow_queue
    empty = False
    while not empty:
            try:
                batch = queue.get(timeout=0.5)
            except Empty:
                empty = True
                continue

            batch = queue.get()
            data = batch[0]
            labels = batch[1]
            model = model(weights)
            trained = model.fit(data,lables,epochs)
            weights = trained.weights()
    acc = trained.evaluate(images_val,lables_val)
    return (trained.weights(), acc)



In [35]:
from dask_ml.wrappers import Incremental
cnn = build_cnn()

dask_cnn = Incremental(cnn)
dask_cnn

dask_cnn.fit(images_training, labels_training)








Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 196, 196, 30)      840       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 19, 19, 30)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 17, 30)        8130      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 1, 1, 30)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 30)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                1550      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                

TypeError: Cannot clone object '<tensorflow.python.keras.engine.sequential.Sequential object at 0x7f6a886ea670>' (type <class 'tensorflow.python.keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [97]:
from keras import initializers
import tensorflow as tf

In [115]:
epochs = 1
iterations = 500
model = build_cnn()

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=101010) 


#Initialize all the weights. The initialized weights are stored in a list OF arrays.
#(each element of the list is an array containing the weights of that layer)

weights_initial = []
for layer in model.layers:
    name = model.get_layer(layer.name)
    name.set_weights([initializer(shape=w.shape) for w in layer.get_weights()])
    weights_initial.append(name.get_weights())    

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 196, 196, 30)      840       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 19, 19, 30)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 17, 17, 30)        8130      
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 1, 1, 30)          0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 30)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 50)                1550      
_________________________________________________________________
dense_33 (Dense)             (None, 2)               

In [116]:
for i in range(iterations):
    trainedWeightsCollection = client.map(trainFunction, batches, **{'model':model,'weights':weights_initial, 'epochs':epochs,'images_val':images_val,'lables_val':lables_val})
    acc = []
    weights = []
    for results in trainedWeightsCollection:
        acc.append(results)
        weights.append(results)
    bestIndex = np.argmax(acc)
    weights = weightsList[bestIndex]
    
#collect a list of accuracies at each itteration for plotting.
#in same plot show marco's versions as epochs for comparison. Dask should get higher accuracies faster.

TypeError: '>' not supported between instances of 'Future' and 'Future'

distributed.protocol.pickle - INFO - Failed to serialize {'model': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fb16eb78610>, 'weights': [[array([[[[-8.66327714e-03,  1.65887959e-02, -4.62062433e-02,
           2.41889041e-02, -1.59043912e-02, -3.32188569e-02,
          -8.52061138e-02,  2.88227778e-02, -6.59869835e-02,
          -3.64983231e-02, -1.48401698e-02, -4.17027175e-02,
           2.33296435e-02,  6.45364309e-03,  1.34234941e-02,
          -7.06457580e-03,  3.32827680e-03,  8.34245384e-02,
           1.35747362e-02,  5.77140450e-02,  1.32656738e-01,
           1.56748772e-01, -6.54316917e-02,  2.44819298e-02,
          -3.29254493e-02,  5.37278354e-02,  4.10328619e-02,
          -5.68654947e-02,  9.12895630e-05,  2.75620054e-02],
         [ 6.92711864e-03, -2.76636574e-02,  3.32103483e-02,
           3.86664271e-02,  2.80784052e-02, -1.10298460e-02,
           5.30900024e-02, -3.13320160e-02,  9.45096742e-03,
           4.16437238e-02, -5.54055348e-02, -

distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/distributed/protocol/core.py", line 76, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
  File "/opt/conda/lib/python3.9/site-packages/msgpack/__init__.py", line 35, in packb
    return Packer(**kwargs).pack(o)
  File "msgpack/_packer.pyx", line 294, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 300, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 297, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_pa

distributed.comm.utils - INFO - Unserializable Message: [{'op': 'update-graph-hlg', 'hlg': {'layers': [{'__module__': 'dask.highlevelgraph', '__name__': 'MaterializedLayer', 'state': {'dsk': {'trainFunction-09190630cecbe625961905123a60fdd5': <Serialize: (<function apply at 0x7fb1a06d7b80>, <function trainFunction at 0x7fb19c6e40d0>, (<class 'tuple'>, [Delayed('list-bbde3234-1621-450f-99bf-9546095241ca')]), {'model': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fb16eb78610>, 'weights': [[array([[[[-8.66327714e-03,  1.65887959e-02, -4.62062433e-02,
           2.41889041e-02, -1.59043912e-02, -3.32188569e-02,
          -8.52061138e-02,  2.88227778e-02, -6.59869835e-02,
          -3.64983231e-02, -1.48401698e-02, -4.17027175e-02,
           2.33296435e-02,  6.45364309e-03,  1.34234941e-02,
          -7.06457580e-03,  3.32827680e-03,  8.34245384e-02,
           1.35747362e-02,  5.77140450e-02,  1.32656738e-01,
           1.56748772e-01, -6.54316917e-02,  2.44819298e-02

distributed.comm.utils - ERROR - ('Could not serialize object of type dict.', "{'model': <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fb16eb78610>, 'weights': [[array([[[[-8.66327714e-03,  1.65887959e-02, -4.62062433e-02,\n           2.41889041e-02, -1.59043912e-02, -3.32188569e-02,\n          -8.52061138e-02,  2.88227778e-02, -6.59869835e-02,\n          -3.64983231e-02, -1.48401698e-02, -4.17027175e-02,\n           2.33296435e-02,  6.45364309e-03,  1.34234941e-02,\n          -7.06457580e-03,  3.32827680e-03,  8.34245384e-02,\n           1.35747362e-02,  5.77140450e-02,  1.32656738e-01,\n           1.56748772e-01, -6.54316917e-02,  2.44819298e-02,\n          -3.29254493e-02,  5.37278354e-02,  4.10328619e-02,\n          -5.68654947e-02,  9.12895630e-05,  2.75620054e-02],\n         [ 6.92711864e-03, -2.76636574e-02,  3.32103483e-02,\n           3.86664271e-02,  2.80784052e-02, -1.10298460e-02,\n           5.30900024e-02, -3.13320160e-02,  9.45096742e-03,\n          

distributed.batched - ERROR - Error in batched write
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/distributed/batched.py", line 94, in _background_send
    nbytes = yield self.comm.write(
  File "/opt/conda/lib/python3.9/site-packages/tornado/gen.py", line 762, in run
    value = future.result()
  File "/opt/conda/lib/python3.9/site-packages/distributed/comm/tcp.py", line 256, in write
    frames = await to_frames(
  File "/opt/conda/lib/python3.9/site-packages/distributed/comm/utils.py", line 70, in to_frames
    return _to_frames()
  File "/opt/conda/lib/python3.9/site-packages/distributed/comm/utils.py", line 53, in _to_frames
    return list(protocol.dumps(msg, **kwargs))
  File "/opt/conda/lib/python3.9/site-packages/distributed/protocol/core.py", line 76, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
  File "/opt/conda/lib/python3.9/site-packages/msgpack/__init__.py", line 35, in packb
    return Packe

In [121]:
for i in range(len(acc)): 
    print(acc[i].result())

CancelledError: trainFunction-09190630cecbe625961905123a60fdd5